In [34]:
import json
import yaml
import glob
import os
import numpy as np
import sys
import shutil

sys.path.insert(0, "../../hindsight/hindsight_server/")
from db import HindsightDB

In [14]:
project = "twitter-2024-09-13-20-23-70120e04_with_negs"
project_dir = os.path.abspath(f"../data/label_studio/{project}")

In [15]:
# Create train.config.yaml
config_d = {"path" : project_dir, "train" : "images", "val" : "images"}
with open(os.path.join(project_dir, "notes.json"), 'r') as infile:
    project_notes = json.load(infile)

id_to_entity = {}
for cat in project_notes['categories']:
    id_to_entity[cat['id']] = cat['name']

config_d["names"] = id_to_entity

config_f = os.path.join(project_dir, "train_config.yaml")
with open(config_f, 'w') as outfile:
    yaml.dump(config_d, outfile)

In [16]:
# Scrollabel cleanup x labels to full width of image
labels_dir = os.path.join(project_dir, "labels")
for f in os.listdir(labels_dir):
    labels_file_path = os.path.join(labels_dir, f)
    new_label_data = list()
    with open(labels_file_path, 'r') as infile:
        for line in infile.readlines():
            line_s = line.split(" ")
            if len(line_s) != 1:
                if int(line_s[0]) in {0, 4, 5, 6, 10, 11, 12, 13}:
                    new_line = " ".join([line_s[0], str(0.5), line_s[2], str(1), line_s[4]])
                    new_label_data.append(new_line)
                else:
                    new_label_data.append(line)

    with open(labels_file_path, 'w') as outfile:
        for new_line in new_label_data:
            outfile.write(new_line)

# Add random screenshots as negatives

In [18]:
db = HindsightDB()

In [28]:
frames = db.get_frames(impute_applications=False)
frames = frames.loc[~frames['application'].isin(["Twitter", "backCamera", "frontCamera"])]

In [29]:
num_positive_samples = len(os.listdir(labels_dir))

In [30]:
frames = frames.sample(num_positive_samples)

In [35]:
images_dir = os.path.join(project_dir, "images")
for i, row in frames.iterrows():
    filename = os.path.basename(row['path'])
    im_dest = os.path.join(images_dir, filename)
    shutil.copy(row['path'], im_dest)

    label_dest = os.path.join(labels_dir, filename)
    with open(label_dest, 'w') as outfile:
        outfile.write("")

# Train

In [36]:
import os
import sys
import time
import datetime
import argparse

import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torch.autograd import Variable
import torch.optim as optim

In [37]:
from ultralytics import YOLO

In [38]:
model = YOLO("yolov8m.pt")

In [40]:
results = model.train(data=config_f, epochs=10, device="mps", rect=True, augment=False, close_mosaic=0, batch=15, scale=0, translate=0)

New https://pypi.org/project/ultralytics/8.2.94 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.91 🚀 Python-3.10.14 torch-2.5.0.dev20240625 MPS (Apple M3 Max)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/Users/connorparish/code/hindsight_parsing/data/label_studio/twitter-2024-09-13-20-23-70120e04_with_negs/train_config.yaml, epochs=10, time=None, patience=100, batch=15, imgsz=640, save=True, save_period=-1, cache=False, device=mps, workers=0, project=None, name=train162, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=True, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment

train: Scanning /Users/connorparish/code/hindsight_parsing/data/label_studio/twitter-2024-09-13-20-23-70120e04_with_negs/labels.cache... 75 images, 77 backgrounds, 0 corrupt: 100%|██████████| 150/150 [00:00<?, ?it/s]

WARNING ⚠️ 'rect=True' is incompatible with DataLoader shuffle, setting shuffle=False



val: Scanning /Users/connorparish/code/hindsight_parsing/data/label_studio/twitter-2024-09-13-20-23-70120e04_with_negs/labels.cache... 75 images, 77 backgrounds, 0 corrupt: 100%|██████████| 150/150 [00:00<?, ?it/s]

Plotting labels to runs/detect/train162/labels.jpg... 


...

                   all        150        284      0.835      0.691      0.906       0.81

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      10/10         0G    0.08924     0.3353     0.4314          0        320: 100%|██████████| 10/10 [00:08<00:00,  1.18it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:04<00:00,  1.00it/s]


                   all        150        284      0.861      0.671      0.923      0.825

10 epochs completed in 0.091 hours.
Optimizer stripped from runs/detect/train162/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/train162/weights/best.pt, 52.0MB

Validating runs/detect/train162/weights/best.pt...
Ultralytics YOLOv8.2.91 🚀 Python-3.10.14 torch-2.5.0.dev20240625 MPS (Apple M3 Max)
Model summary (fused): 218 layers, 25,847,866 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:06<00:00,  1.24s/it]


                   all        150        284      0.861      0.672      0.923      0.825
discover_new_communities          3          3       0.54      0.414      0.641      0.572
           more_posted         25         25      0.958          1      0.995      0.964
             new_posts          3          3      0.763          1      0.995      0.995
         partial_tweet          1          2          1          0      0.995      0.895
pinned_by_people_you_follow          1          1          1          0      0.995      0.995
         playing_video          5          6          1      0.962      0.995      0.953
           plus_button         15         15          1          1      0.995       0.97
          quoted_tweet          1          1          1          0      0.995      0.995
      subscribe_button          1          1      0.532          1      0.995      0.995
                 tweet         72        189      0.925      0.995      0.992      0.906
              